In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

# HearLight libraries

In [5]:
from pynq.lib import MicroblazeLibrary

lib = MicroblazeLibrary(base.ARDUINO, ['spi', 'gpio'])

In [6]:
# enable external power supply with en pin connected to A6
power_supply_en = lib.gpio_open(19)

power_supply_en.set_direction(lib.GPIO_OUT)
power_supply_en.write(1)

# RPC handle to control DACs through SPI
dac = lib.spi_open(13, 12, 11, 17)

# chip select for DAC A and DAC B - initialise to 'high' (slave disabled)
cs_a = lib.gpio_open(10)
cs_b = lib.gpio_open(2)

cs_a.set_direction(lib.GPIO_OUT)
cs_a.write(1)

cs_b.set_direction(lib.GPIO_OUT)
cs_b.write(1)

# RPC handle for switches control through GPIO - initialise to 'high' (switch open)
switches = [lib.gpio_open(3),
            lib.gpio_open(4),
            lib.gpio_open(5),
            lib.gpio_open(6),
            lib.gpio_open(7),
            lib.gpio_open(8),
            lib.gpio_open(9),
            lib.gpio_open(14),
            lib.gpio_open(15),
            lib.gpio_open(16)]

for sw in switches:
    sw.set_direction(lib.GPIO_OUT)
    sw.write(1) # low for close switch

In [7]:
LTC2662_CMD_POWER_DOWN_N = 0x40
LTC2662_CMD_SPAN = 0x60
LTC2662_CMD_WRITE_N_UPDATE_N = 0x30

# function to transfer block through SPI
def dac_write(cs_handle, dac_command, selected_dac, code):
    tx_array = bytearray(4)
    rx_array = bytearray(4)
    
    dac_code = bytearray(2)
    dac_code[:] = bytearray(int.to_bytes(code, 2, 'little'))
    
    tx_array[0] = 0 # first byte is zero
    tx_array[1] = dac_command | selected_dac # command is upper 4 bits, selected DAC is lower 4 bits
    tx_array[2] = dac_code[1]
    tx_array[3] = dac_code[0]
    
    cs_handle.write(0)
    dac.transfer(tx_array, rx_array, 4) # 4 bytes data to read
    cs_handle.write(1)
    
    fault_reg = rx_array[0]
    
    return fault_reg

In [8]:
def switch_control(sw, open_close):
    """ Opens/closes switches in the switch arrays based on 'sw' argument
        'open_close' = '1' is switch closed and '0' is switch open
    """
    if(open_close):
        switches[sw-1].write(0)
    else:
        switches[sw-1].write(1)

def dac_config(channel, current_ref):
    """ sets softspan range of all DACs based on 'current_ref'
    """
    fault_reg = 0
    
    selected_dac = (channel - 1) % 5
    
    user_command = 0
    
    if(current_ref == 0):
        user_command = 1
    elif(current_ref == 1):
        user_command = 2
    elif(current_ref == 2):
        user_command = 3
    elif(current_ref == 3):
        user_command = 4
    elif(current_ref == 4):
        user_command = 5
    elif(current_ref == 5):
        user_command = 6
        
    if(channel > 0 and channel <= 5):
        fault_reg |= dac_write(cs_a, LTC2662_CMD_SPAN, selected_dac, user_command)
    elif(channel > 5 and channel <= 10):
        fault_reg |= dac_write(cs_b, LTC2662_CMD_SPAN, selected_dac, user_command)
        
    return fault_reg
        
def dac_channel_control(channel, on_off, current_code):
    """ Switches on/off DAC channel indicated by 'channel' argument and considers both DACs
        'channel' => 1->10
        'on_off' => '1' is on and '0' is off
        'current_code' => current value in DAC counts
    """
    fault_reg = 0
    
    selected_dac = (channel - 1) % 5
    
    if(current_code == 0):
        on_off = 0
        
    dac_code = current_code
    
    if(on_off):
        if(channel > 0 and channel <= 5):
            fault_reg |= dac_write(cs_a, LTC2662_CMD_WRITE_N_UPDATE_N, selected_dac, dac_code)
        elif(channel > 5 and channel <= 10):
            fault_reg |= dac_write(cs_b, LTC2662_CMD_WRITE_N_UPDATE_N, selected_dac, dac_code)            
    else:
        if(channel > 0 and channel <= 5):
            fault_reg |= dac_write(cs_a, LTC2662_CMD_POWER_DOWN_N, selected_dac, 0)
        elif(channel > 5 and channel <= 10):
            fault_reg |= dac_write(cs_b, LTC2662_CMD_POWER_DOWN_N, selected_dac, 0)
            
    return fault_reg

In [ ]:
# ALL SWITCHES OPEN
for i in range(10):
    switch_control(i+1, 0)

In [9]:
# SWITCH OFF ALL DAC CHANNELS
for i in range(10):
    dac_channel_control(i+1, 0, 0)

In [10]:
# SET SOFTSPAN RANGE FOR ALL DACS
for i in range(10):
    dac_config(i+1, 5)

# Audio processing

In [11]:
pAudio = base.audio
pAudio.set_volume(20)

In [12]:
pAudio.select_line_in()

In [30]:
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import ipywidgets as widgets

time_to_run = 20
sample_time = 0.2
buffer_len = int(sample_time * 48000 * 2)

# time domain plot
buffer_signed = np.zeros(buffer_len)

line = px.line(x = np.arange(0, buffer_len, 1),
               y = buffer_signed)

fig = go.FigureWidget(line, layout = go.Layout(xaxis = {'title' : 'sample'},
                                      yaxis = {'title' : 'amplitude'}))
fig.update_yaxes(
    range=(-250000, 250000),
    constrain='domain'
)

# freq domain
buffer_ft = np.zeros(buffer_len)

ft_mag_line = px.line(x = np.arange(0, buffer_len, 1),
                      y = buffer_ft)

fig_ft = go.FigureWidget(ft_mag_line)

widgets.HBox(children=[fig, fig_ft])

    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              …

In [31]:
uLED_freq_bins = np.arange(30, 3030, 30).reshape(10,10)

for i in range(int(time_to_run / sample_time)):
    pAudio.record(seconds=sample_time)
    buffer_signed = (pAudio.buffer << 8).view(np.int32) >> 8
    
    fig.data[0].update({'y' : buffer_signed})
    
    # compute fft
    ft = abs(np.fft.fft(buffer_signed))
    fig_ft.data[0].update({'y' : buffer_signed})
    
    # get frequency
    idx = np.where(ft == np.amax(ft[0:int(len(ft)/2)]))[0][0]
    freq = np.fft.fftfreq(19200, d=1/48000)[idx] * 2 # times by 2...I think this is to do with not separating stereo channels..
    print(freq)
    
    # control uLEDs
    row = np.where(abs(uLED_freq_bins-freq) == np.amin(abs(uLED_freq_bins-freq)))[0][0]
    col = np.where(abs(uLED_freq_bins-freq) == np.amin(abs(uLED_freq_bins-freq)))[1][0]

    # SWITCH OFF ALL DAC CHANNELS
    for i in range(10):
        dac_channel_control(i+1, 0, 0)
    # ALL SWITCHES OPEN
    for i in range(10):
        switch_control(i+1, 0)
        
    switch_control(row+1, 1)
    dac_channel_control(col+1, 1, 2048)

# SWITCH OFF ALL DAC CHANNELS
for i in range(10):
    dac_channel_control(i+1, 0, 0)
# ALL SWITCHES OPEN
for i in range(10):
    switch_control(i+1, 0)

140.0
0.0
90.0
5.0
155.0
150.0
130.0
140.0
130.0
50.0
0.0
5.0
0.0
50.0
0.0
115.0
120.0
130.0
50.0
0.0
5.0
0.0
50.0
350.0
130.0
125.0
130.0
130.0
130.0
120.0
125.0
130.0
275.0
280.0
280.0
275.0
285.0
495.0
440.0
440.0
0.0
5.0
0.0
0.0
130.0
130.0
130.0
0.0
245.0
0.0
5.0
0.0
95.0
200.0
50.0
0.0
50.0
0.0
0.0
5.0
130.0
50.0
125.0
130.0
0.0
95.0
0.0
5.0
0.0
0.0
0.0
0.0
5.0
0.0
50.0
0.0
5.0
0.0
50.0
0.0
125.0
130.0
0.0
0.0
0.0
5.0
0.0
130.0
135.0
5.0
0.0
200.0
0.0
5.0
0.0
240.0
0.0
110.0
200.0
615.0
